In [ ]:
# Code written by Ji Zhang, last modified 2024.01.11
# For manuscript "Controllable Multimodal Actuation in Fully Printed Ultrathin Micro-Patterned Electrochemical Actuators"
# email: zhangji1623316718@gmail.com

import pandas as pd
import os
import numpy as np
import scipy as sp
from scipy import integrate
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import MultipleLocator
def average(lst):
  return sum(lst)/len(lst)
def sd(lst):
  n = len(lst)
  m = sum(lst)/n
  var = sum((np.array(lst)-m)**2)/n
  return np.sqrt(var)
def sds(lst):
  n = len(lst)
  m = sum(lst)/n
  var = sum((np.array(lst)-m)**2)/(n-1)
  return np.sqrt(var)
class my_dictionary(dict):
  def __init__(self):
    self = dict()
  # Function to add key:value
  def add(self, key, value):
    self[key] = value

from scipy.signal import find_peaks

def peakfinder(x,y,height,distance):
  # Function adapted from Liam Ives
  xdata = x
  ydata = y
  # Find indices of peaks and troughs
  peak_indices, peak_properties = find_peaks(ydata, height=height,distance=distance)
  trough_indices, trough_properties = find_peaks(-ydata, height=-height,distance=distance)  #changed height=-height*0.6
  # Convert peak indices into time values
  peak_times = []
  for peak in peak_indices:
    peak_time = xdata[peak] # find peak time from index given in 'peaks'
    peak_times.append(peak_time)
  trough_times = []
  for trough in trough_indices:
    trough_time = xdata[trough] # find peak time from index given in 'peaks'
    trough_times.append(trough_time)
  print('Finding peak times...')
  print(peak_times)
  print('Finding trough times...')
  print(trough_times)
  return peak_indices, peak_times, trough_indices, trough_times, xdata, ydata

# Plot rc parameters adapted from Tom Wade
fig, ax = plt.subplots() # Need to open a 'plot' so that pcParams actually works
plt.close()
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

plt.rcParams.update({
    'font.size': 14,          # controls default text sizes
    'axes.labelsize': 14,     # fontsize of the x and y labels
    'axes.titlesize': 14,     # fontsize of the axes title
    'xtick.labelsize': 14,    # fontsize of the tick labels
    'ytick.labelsize': 14,    # fontsize of the tick labels
    'legend.fontsize': 14,    # fontsize of the legend
    'figure.titlesize': 16,   # fontsize of the figure title
})

current_folder_path = os.getcwd().replace('\\', '/')
print(current_folder_path)

In [5]:
def get_df(folder_IV_path, folder_def_path, startwith):
    for filename1 in os.listdir(folder_def_path):
        # if True:
        if filename1.startswith(startwith):
            filedef = os.path.join(folder_def_path, filename1).replace('\\', '/')
            for filename2 in os.listdir(folder_IV_path):
                if filename2.startswith(filename1[0:4]):
                    fileIV = os.path.join(folder_IV_path, filename2).replace('\\', '/')
                    break
            df1 = pd.read_csv(fileIV, sep='\t', header=None)
            df2 = pd.read_csv(filedef, sep=',', header=None)
            df2[3] = np.arctan(df2[2]/df2[1])
            break
    return df1, df2

def read(df1, df2):
    # df1 = pd.read_csv(fileIV, sep='\t', header=None)
    # df2 = pd.read_csv(filedef, sep=',', header=None)
    ti = df1[0].to_numpy()
    I = df1[2].to_numpy()
    V = df1[3].to_numpy()
    td = df2[0].to_numpy()
    # dx = df2[1].to_numpy()
    # dy = df2[2].to_numpy()
    # d = np.arctan(dy/dx)
    d = df2[3].to_numpy()
    return ti, I, V, td, d

def plot_V_I_def(ti, I, V, td, d):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(20,8), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3]))

    ax1.plot(ti, V, color='tab:orange', lw=1)
    ax1.set_ylabel('$V$/V', color='black')
    # ax2.scatter(ti, I, color='blue', marker='o', edgecolors='blue', s=10, alpha=0.5, facecolors="none")
    # ax3.scatter(td, d, color='green', marker='s', edgecolors='green', s=10, alpha=0.5, facecolors="none")
    ax2.plot(ti, I, color='blue', lw=1)
    ax2.set_ylabel('$I$/mA', color='black')
    ax3.plot(td, d, color='green', lw=1)
    
    ax1.axhline(y=0, color='#999999', linestyle='--')
    ax2.axhline(y=0, color='#999999', linestyle='--')

    ax3.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
    ax3.xaxis.label.set_color('black')
    ax3.yaxis.label.set_color('black')

    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)

    ax3.axhline(y=d[3], color='#999999', linestyle='--')
    # remove last tick label for the second subplot
    yticks = ax1.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)

    ax1.tick_params(direction='out', length=5, width=1, colors='black')
    ax2.tick_params(direction='out', length=5, width=1, colors='black')
    ax3.tick_params(direction='out', length=5, width=1, colors='black')

    ax1.grid(axis = 'x', which='minor')
    ax2.grid(axis = 'x', which='minor')
    ax3.grid(axis = 'x', which='minor')
    ax1.grid(axis = 'x', which='major')
    ax2.grid(axis = 'x', which='major')
    ax3.grid(axis = 'x', which='major')
    return fig, (ax1, ax2, ax3)

def scatter_V_I_def(ti, I, V, td, d):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(14,7), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3, 3]))

    # ax1.scatter(ti, V, color='tab:orange', marker='o', edgecolors='tab:orange', s=10, alpha=0.3, facecolors="none")
    ax1.plot(ti, V, color='tab:orange', lw=1)
    ax1.set_ylabel('$V$/V', color='black')
    ax2.scatter(ti, I, color='blue', marker='o', edgecolors='blue', s=10, alpha=0.3, facecolors="none")
    ax2.set_ylabel('$I$/mA', color='black')
    
    ax1.axhline(y=0, color='#999999', linestyle='--')
    ax2.axhline(y=0, color='#999999', linestyle='--')
    ax3.axhline(y=0, color='#999999', linestyle='--')
    ax3.set_ylabel(r'$Q_{fit}$/mC', color='black')
    
    ax4.scatter(td, d, color='green', marker='s', edgecolors='green', s=10, alpha=0.3, facecolors="none")
    ax4.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
    ax4.xaxis.label.set_color('black')
    ax4.yaxis.label.set_color('black')
    
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)

    ax4.axhline(y=d[1], color='#999999', linestyle='--')
    # remove last tick label for the second subplot
    yticks = ax1.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax2.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax3.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)

    ax1.tick_params(direction='out', length=5, width=1, colors='black')
    ax2.tick_params(direction='out', length=5, width=1, colors='black')
    ax3.tick_params(direction='out', length=5, width=1, colors='black')
    ax4.tick_params(direction='out', length=5, width=1, colors='black')
    
    return fig, (ax1, ax2, ax3, ax4)

def plot_V_I_Q_def(ti, I, V, td, d):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(20,8), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3, 3]))

    ax1.plot(ti, V, color='tab:orange', lw=1)
    ax1.set_ylabel('$V$/V', color='black')
    ax2.plot(ti, I, color='blue', lw=1)
    ax2.set_ylabel('$I$/mA', color='black')
    Q = sp.integrate.cumulative_trapezoid(I, ti, initial=0)
    ax3.plot(ti, Q, color='tab:blue', lw=1)
    
    ax1.axhline(y=0, color='#999999', linestyle='--')
    ax2.axhline(y=0, color='#999999', linestyle='--')
    ax3.axhline(y=0, color='#999999', linestyle='--')
    ax3.set_ylabel(r'$Q_{fit}$/mC', color='black')
    
    ax4.plot(td, d, color='green', lw=1)
    ax4.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
    ax4.xaxis.label.set_color('black')
    ax4.yaxis.label.set_color('black')
    
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)

    ax4.axhline(y=d[3], color='#999999', linestyle='--')
    # remove last tick label for the second subplot
    yticks = ax1.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax2.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    yticks = ax3.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)

    ax1.tick_params(direction='out', length=5, width=1, colors='black')
    ax2.tick_params(direction='out', length=5, width=1, colors='black')
    ax3.tick_params(direction='out', length=5, width=1, colors='black')
    ax4.tick_params(direction='out', length=5, width=1, colors='black')

    return fig, (ax1, ax2, ax3, ax4), Q

def readSplit(df1, df2, marks):
    
    # marks = [i*interval for i in range(nInterval)]

    df_I = my_dictionary()
    df_d = my_dictionary()
    pd.options.mode.chained_assignment = None
    for i in range(len(marks)-1):
        values_I = df1[(df1[0]>marks[i]) & (df1[0]<marks[i+1])]
        values_d = df2[(df2[0]>marks[i]) & (df2[0]<marks[i+1]-0.12)]
        # print(values_I.iloc[0,2])
        index = values_I.index[0]
        if i!=0:
            for j in range(5):
                if abs(values_I.iloc[0,2])<0.005:
                    values_I.drop([index+j], inplace=True)
                else:
                    break
    
        df_I.add(i, values_I)
        df_d.add(i, values_d)
    timeI = []
    VV=[]
    II=[]
    timed = []
    dd=[]
    for i in range(len(marks)-1):
        timeI.append(df_I[i][0].to_numpy()-marks[i])
        VV.append(df_I[i][3].to_numpy())
        II.append(df_I[i][2].to_numpy())
        timed.append(df_d[i][0].to_numpy()-marks[i])
        dd.append(df_d[i][3].to_numpy())
    return timeI, VV, II, timed, dd

def plot_part(timeI, VV, II, timed, dd, i):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(20,8), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3]))
    ax1.scatter(timeI[i], VV[i], color='tab:orange', marker='o', edgecolors='tab:orange', s=10, alpha=0.3, facecolors="none")
    ax1.set_ylabel('$V$/V', color='black')
    ax2.scatter(timeI[i], II[i], color='blue', marker='o', edgecolors='blue', s=10, alpha=0.3, facecolors="none")
    ax2.set_ylabel('$I$/mA', color='black')
    ax3.scatter(timed[i], dd[i], color='green', marker='s', edgecolors='green', s=10, alpha=0.3, facecolors="none")
    ax3.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
    ax3.xaxis.label.set_color('black')
    ax3.yaxis.label.set_color('black')
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)
    # remove last tick label for the second subplot
    yticks = ax2.yaxis.get_major_ticks()
    yticks[-1].label1.set_visible(False)
    plt.subplots_adjust(hspace=.0)
    ax1.tick_params(direction='out', length=5, width=1, colors='black')
    ax2.tick_params(direction='out', length=5, width=1, colors='black')
    ax3.tick_params(direction='out', length=5, width=1, colors='black')

    ax2.axhline(y=0, color='#999999', linestyle='--')
    # ax3.axhline(y=D, color='#999999', linestyle='--')
    # ax2.plot(timeI[i], Ifit, color='blue', lw=1)
    # ax3.plot(timed[i], dfit, color='green', lw=1)
    return fig, (ax1, ax2, ax3)

In [6]:
def chargeI(x, A1, A2, t1, t2, C): #5 para
    y = A1*np.exp(-x/t1) + A2*np.exp(-x/t2) + C
    return y
# def chargeI(x, A1, A2, t1, t2): #4 para
#     y = A1*np.exp(-x/t1) + A2*np.exp(-x/t2)
#     return y
def chargedef(x, B1, B2, t3, t4, D): #5 para
    y = B1*(1-np.exp(-x/t3)) + B2*(1-np.exp(-x/t4)) + D
    return y
def chargeQ(x, A1, A2, t1, t2):
    y = A1*t1*(1-np.exp(-x/t1)) + A2*t2*(1-np.exp(-x/t2))
    return y

def shortI(x, A1, A2, t1, t2): #4 para
    y = -A1*np.exp(-x/t1) - A2*np.exp(-x/t2)
    return y
def shortdef(x, B1, B2, t3, t4, D): #5 para
    y = B1*np.exp(-x/t3) + B2*np.exp(-x/t4) + D
    return y
def shortQ(x, A1, A2, t1, t2):
    y = -A1*t1*(1-np.exp(-x/t1)) - A2*t2*(1-np.exp(-x/t2))
    return y

def twoExp(x, A1, t1, t2): #3 para
    y = A1*np.exp(-x/t1) + (0.6-A1)*np.exp(-x/t2)
    return y

In [ ]:
## PEDOT:PSS_conductivity (Figure S3)
def linprop(x, k):
    return k*x

h = np.array([1.1, 2.1, 3.2]) #e-6
R = np.array([95, 47, 25]) #e3
G = 1/R #mS
param, cov = curve_fit(linprop, h, G, p0=[0.0125])
print(param[0])

fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_xlabel('PEDOT:PSS thickness/μm')
ax.set_ylabel('Conductance/mS')
ax.plot([0,3.5], [0,3.5*param[0]])
ax.scatter(h,G, color='black', s=20)
ax.set_xlim(0)
ax.set_ylim(0)

ax.yaxis.set_major_locator(MultipleLocator(0.01))
ax.yaxis.set_minor_locator(MultipleLocator(0.005))
ax.xaxis.set_minor_locator(MultipleLocator(0.5))

# plt.savefig('conductivity.svg',format='svg')

# conductivity = gradient * l/w [mS/um*mm/mm]
print(param[0]*3/0.7) # S/mm

In [ ]:
## Sample II_CV (Figure S4 and Figure 3C)

folder_IV_path = f"{current_folder_path}/Sample II_CV/IV"
folder_def_path = f"{current_folder_path}/Sample II_CV/def"

def plotCV(startwith):
    df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(startwith))
    ti, I, V, td, d = read(df1, df2)
    fig, (ax1, ax2, ax3, ax4),Q = plot_V_I_Q_def(ti, I, V, td, d)
    ax1.text(0,0.1, f'{startwith}')
    ax4.set_xlim(0)
    return fig, ax2

fig, ax2 = plotCV(6391)
ax2.set_ylim(-0.027)
# fig.savefig('6391.svg', format='svg')
fig,ax2 = plotCV(6392)
ax2.set_ylim(-0.02)
# fig.savefig('6392.svg', format='svg')
fig,ax2 = plotCV(6395)
ax2.set_ylim(-0.01)
# fig.savefig('6395.svg', format='svg')
plt.show()

df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(6391))
ti, I, V, td, d = read(df1, df2)
plt.plot(V[1000:],I[1000:],label='100 mV/s')
df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(6392))
ti, I, V, td, d = read(df1, df2)
plt.plot(V[1000:],I[1000:], label = '50 mV/s')
df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(6395))
ti, I, V, td, d = read(df1, df2)
plt.plot(V[1000:],I[1000:],label='20 mV/s')
plt.xlabel('Voltage/V')
plt.ylabel('Current/mA')
plt.legend()
# plt.savefig('CV.svg', format='svg')

In [ ]:
def plotCV(startwith):
    df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(startwith))
    ti, I, V, td, d = read(df1, df2)
    fig, (ax1, ax2, ax3, ax4), Q = plot_V_I_Q_def(ti, I, V, td, d)
    # ax1.text(0,0.1, f'{startwith}')
    ax4.set_xlim(0)
    # plt.show()
    # plt.plot(V[1000:],I[1000:])
    # plt.ylim(-0.006, 0.006)
    return fig, ax2,ax3, ti, Q

for startwith in [6391, 6392, 6395]:
    fig, ax2,ax3, ti, Q = plotCV(startwith)
    peak_def_indices, peak_def_times, trough_def_indices, trough_def_times, xdefdata, ydefdata = peakfinder(ti,Q, height=0, distance = 500)
    peak_def_values = ydefdata[peak_def_indices]
    trough_def_values = ydefdata[trough_def_indices]
    front = 1
    back = 0
    Q_pp = average(peak_def_values[front:(len(peak_def_values)-back)]) - average(trough_def_values[front:(len(peak_def_values)-back)])
    sd_Q_pp = np.sqrt(sd(peak_def_values[front:(len(peak_def_values)-back)])**2 + sd(trough_def_values[front:(len(peak_def_values)-back)])**2)
    print(Q_pp, sd_Q_pp)

    ax3.scatter(peak_def_times,peak_def_values)
    ax3.scatter(trough_def_times,trough_def_values)

In [ ]:
## DC chronoamperometry (Figure 5 and Figure S9)

folder_IV_path = f"{current_folder_path}/Sample II_Actuation DC chronoamperometry/IV"
folder_def_path = f"{current_folder_path}/Sample II_Actuation DC chronoamperometry/def"

def fitTimesAmps(startwith):
    df1, df2 = get_df(folder_IV_path, folder_def_path, startwith=str(startwith))
    ti, I, V, td, d = read(df1, df2)

    marks = [i*50 for i in range(16)]
    timeI, VV, II, timed, dd = readSplit(df1, df2, marks)
    df = pd.DataFrame(columns=['V', 'A1', 'A2', 't1', 't2', 'C', 'B1', 'B2', 't3', 't4', 'D', 'Qp'])

    fig, (ax1, ax2, ax3, ax4) = scatter_V_I_def(ti, I, V, td, d)
    ax4.set_xlim(0,750)
    ax1.yaxis.set_major_locator(MultipleLocator(0.4))
    ax1.yaxis.set_minor_locator(MultipleLocator(0.2))
    ax2.yaxis.set_major_locator(MultipleLocator(0.2))
    ax2.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax3.yaxis.set_major_locator(MultipleLocator(0.02))
    # ax3.yaxis.set_minor_locator(MultipleLocator(0.02))
    ax3.xaxis.set_major_locator(MultipleLocator(50))
    ax3.xaxis.set_minor_locator(MultipleLocator(10))

    Vs=[0.2, 0.4, 0.6, 0.8, 0.6, 0.4, 0.2]

    for i in [1, 3, 5, 7, 9, 11, 13]:
        def fitI(x, A1, A2, t1, t2, C):
            x1 = x[:len(timeI[i])]
            x2 = x[len(timeI[i]):]
            y1 = chargeI(x1, A1, A2, t1, t2, C)
            y2 = shortI(x2, A1, A2, t1, t2)
            return np.concatenate((y1,y2))
        def fitdez(x, B1, B2, t3, t4, D):
            x1 = x[:len(timed[i])]
            x2 = x[len(timed[i]):]
            y1 = chargedef(x1, B1, B2, t3, t4, D)
            y2 = shortdef(x2, B1, B2, t3, t4, D)
            return np.concatenate((y1,y2))

        k = len(df)
        VV = Vs[k]
        paraI, covI = curve_fit(fitI, np.concatenate((timeI[i], timeI[i+1])), np.concatenate((II[i], II[i+1])), p0=[0.1*VV, 0.05*VV, 0.3, 2.5, 0.0004])
        parad, covd = curve_fit(fitdez, np.concatenate((timed[i], timed[i+1])), np.concatenate((dd[i], dd[i+1])), p0=[0.3*VV, 0.2*VV, 1, 5, -0.14])

        A1, A2, t1, t2, C = paraI
        B1, B2, t3, t4, D = parad
        Q0 = 0

        times = np.arange(0,50,0.2)
        Ifit = chargeI(times, A1, A2, t1, t2,C)
        dfit = chargedef(times, B1, B2, t3, t4, D)
        Qfit = chargeQ(times, A1, A2, t1, t2) + Q0
        Qp = Qfit[-1]

        Ifit2 = shortI(times, A1, A2, t1, t2)
        dfit2 = shortdef(times, B1, B2, t3, t4, D) 
        Qfit2 = shortQ(times, A1, A2, t1, t2) + Qp

        df.loc[k] = [VV, A1, A2, t1, t2, C*100, B1, B2, t3, t4, D, Qp]

        ax2.plot(times+marks[i], Ifit, color='blue', lw=1)
        ax2.plot(times+marks[i+1], Ifit2, color='blue', lw=1)
        ax3.plot(times+marks[i], Qfit, color='tab:blue', lw=1)
        ax3.plot(times+marks[i+1], Qfit2, color='tab:blue', lw=1)
        ax4.plot(times+marks[i], dfit, color='green', lw=1)
        ax4.plot(times+marks[i+1], dfit2, color='green', lw=1)

    # pd.options.display.float_format = "{:.2f}".format
    print(startwith)
    print(df)
    # df.to_csv(f'{startwith}output.csv', index=False)
    # plt.savefig(f'{startwith}fittt.png',format='png',dpi=200)
    return df

df = fitTimesAmps(6387)
# dfpara.to_csv('params.csv', index=False)

In [ ]:
## Au IPMC_CV (Figure S11D)

fileCV = f'{current_folder_path}/Au IPMC_Actuation AC and CV/Au_CV60s.csv'
df_CV_all = pd.read_csv(fileCV, skiprows=1, names=['t', 'V', 'I'], encoding='iso-8859-1') #V/V, I/mA

# df_CV = df_CV_all[(df_CV_all['t']>11.03)&(df_CV_all['t']<196)]

df_CV = df_CV_all
V = df_CV['V'].to_numpy()
I = df_CV['I'].to_numpy()

fig, ax = plt.subplots()
fig.set_figwidth(6)
fig.set_figheight(4)

ax.plot(V, I, color='tab:blue', lw=1, alpha=1)

ax.yaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.xaxis.set_minor_locator(MultipleLocator(1))

# ax.set_ylim(0,0.25)
# ax.set_xlim(0,7.3)
ax.set_xlabel('Voltage/V')
ax.set_ylabel('Current/mA')
# ax.axvspan(0, 2, color='y', alpha=0.2, lw=0)

ax.arrow(2, 0.5, 0, .5, width = 0.01, head_width = 0.25, head_length = 0.12, color='black')
# plt.annotate(s='', xy=(2,1), xytext=(0,0.2), arrowprops=dict(arrowstyle='<->'))

# ax.get_figure().savefig('AuCV60s.svg',format='svg')

In [4]:
## Au IPMC actuation (Figure S11A, B, C)

fileVI1 = f'{current_folder_path}/Au IPMC_Actuation AC and CV/Au_I1.csv'
filed1 = f'{current_folder_path}/Au IPMC_Actuation AC and CV/Au_d1.csv'
fileVI2 = f'{current_folder_path}/Au IPMC_Actuation AC and CV/Au_I2.csv'
filed2 = f'{current_folder_path}/Au IPMC_Actuation AC and CV/Au_d2.csv'
df_VI1 = pd.read_csv(fileVI1, skiprows=1, names=['ti', 'V','I'], encoding='iso-8859-1')
df_d1 = pd.read_csv(filed1, skiprows=1, names=['td', 'd'], encoding='iso-8859-1')
df_VI2 = pd.read_csv(fileVI2, skiprows=1, names=['ti', 'V','I'], encoding='iso-8859-1')
df_d2 = pd.read_csv(filed2, skiprows=1, names=['td', 'd'], encoding='iso-8859-1')

ti = df_VI1['ti'].to_numpy() - 11.1
V = df_VI1['V'].to_numpy()
I = df_VI1['I'].to_numpy()
td = df_d1['td'].to_numpy() - 11.1
d = df_d1['d'].to_numpy()

ti2 = df_VI2['ti'].to_numpy() - 8.0
V2 = df_VI2['V'].to_numpy()
I2 = df_VI2['I'].to_numpy()
td2 = df_d2['td'].to_numpy() - 8.0
d2 = df_d2['d'].to_numpy()

In [ ]:
## Au IPMC actuation (Figure S11A (left), B)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(7.5,6), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3]))

ax1.plot(ti, V, color='tab:orange', lw=1)
# ax1.scatter(ti-0.05, V, color='tab:orange', marker='o', edgecolors='tab:orange', s=10, alpha=0.5, facecolors="none")
# ax1.set_ylim(-0.1,0.6)
ax1.set_ylabel('$V$/V', color='black')
# ax2.scatter(ti-0.0, I, color='blue', marker='o', edgecolors='blue', s=10, alpha=0.5, facecolors="none")
# ax3.scatter(td-0.15, d, color='green', marker='s', edgecolors='green', s=10, alpha=0.5, facecolors="none")
ax2.plot(ti, I, color='blue', lw=1)
# ax2.set_ylim(-6,50)
ax2.set_ylabel('$I$/mA', color='black')
ax3.plot(td, d, color='green', lw=1)
# ax3.set_ylim(0.21,0.31)

ax3.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
ax3.xaxis.label.set_color('black')
ax3.yaxis.label.set_color('black')

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)

# ax3.axhline(y=0.223, color='#999999', linestyle='--')
# remove last tick label for the second subplot
yticks = ax1.yaxis.get_major_ticks()
yticks[-1].label1.set_visible(False)
plt.subplots_adjust(hspace=.0)

ax1.tick_params(direction='out', length=5, width=1, colors='black')
ax2.tick_params(direction='out', length=5, width=1, colors='black')
ax3.tick_params(direction='out', length=5, width=1, colors='black')

ax1.set_ylim(-8,8)
ax2.set_ylim(-2.5,2.5)
ax3.set_ylim(-0.26,0.04)

ax2.yaxis.set_major_locator(MultipleLocator(1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.5))

ax3.yaxis.set_major_locator(MultipleLocator(0.1))
ax3.yaxis.set_minor_locator(MultipleLocator(0.05))

ax3.xaxis.set_major_locator(MultipleLocator(100))
ax3.xaxis.set_minor_locator(MultipleLocator(20))
# ax3.xaxis.set_major_locator(MultipleLocator(5))
# ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax1.grid(axis = 'x', which='minor')
ax2.grid(axis = 'x', which='minor')
ax3.grid(axis = 'x', which='minor')
ax1.grid(axis = 'x', which='major')
ax2.grid(axis = 'x', which='major')
ax3.grid(axis = 'x', which='major')

plt.xlim(0,410)
# plt.xlim(370,390)

# fig.get_figure().savefig('AuIPMC1.svg',format='svg')
# fig.get_figure().savefig('backRelax.svg',format='svg')

In [ ]:
## Au IPMC actuation (Figure S11A (right), B)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(5.30475,6), sharex=True, gridspec_kw=dict(height_ratios=[2, 3, 3]))

ax1.plot(ti2, V2, color='tab:orange', lw=1)
ax1.set_ylabel('$V$/V', color='black')
# ax2.scatter(ti, I, color='blue', marker='o', edgecolors='blue', s=10, alpha=0.5, facecolors="none")
# ax3.scatter(td, d, color='green', marker='s', edgecolors='green', s=10, alpha=0.5, facecolors="none")
ax2.plot(ti2, I2, color='blue', lw=1)
ax2.set_ylabel('$I$/mA', color='black')
ax3.plot(td2, d2, color='green', lw=1)

ax3.set(xlabel='Time/s', ylabel=r'$\theta$/rad')
ax3.xaxis.label.set_color('black')
ax3.yaxis.label.set_color('black')

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)

# ax3.axhline(y=0.223, color='#999999', linestyle='--')
# remove last tick label for the second subplot
yticks = ax1.yaxis.get_major_ticks()
yticks[-1].label1.set_visible(False)
plt.subplots_adjust(hspace=.0)

ax1.set_ylim(-8,8)
ax2.set_ylim(-2.5,2.5)
ax3.set_ylim(-0.26,0.04)

ax2.yaxis.set_major_locator(MultipleLocator(1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.5))

ax3.yaxis.set_major_locator(MultipleLocator(0.1))
ax3.yaxis.set_minor_locator(MultipleLocator(0.05))

ax3.xaxis.set_major_locator(MultipleLocator(100))
ax3.xaxis.set_minor_locator(MultipleLocator(20))
ax1.grid(axis = 'x', which='minor')
ax2.grid(axis = 'x', which='minor')
ax3.grid(axis = 'x', which='minor')
ax1.grid(axis = 'x', which='major')
ax2.grid(axis = 'x', which='major')
ax3.grid(axis = 'x', which='major')
plt.xlim(0,290)
# plt.xlim(40,100)

# fig.get_figure().savefig('AuIPMC2.svg',format='svg')

In [ ]:
# finding all peaks and troughs

height = -0.15
distance = 190

peak_indices, peak_properties = find_peaks(d, height=height, distance=distance)
trough_indices, trough_properties = find_peaks(-d, height=-height, distance=distance)
# pk_d = [td[peak] for peak in peak_indices]
# tr_d = [td[trough] for trough in trough_indices]

plt.plot(td,d)
plt.scatter([td[i] for i in peak_indices], [d[i] for i in peak_indices])
plt.scatter([td[i] for i in trough_indices], [d[i] for i in trough_indices])
print([d[i] for i in peak_indices])
print([d[i] for i in trough_indices])
plt.show()

height = -0.1
distance = 190

peak_indices, peak_properties = find_peaks(d2, height=height, distance=distance)
trough_indices, trough_properties = find_peaks(-d2, height=-height, distance=distance)
# pk_d = [td[peak] for peak in peak_indices]
# tr_d = [td[trough] for trough in trough_indices]

plt.plot(td2,d2)
plt.scatter([td2[i] for i in peak_indices], [d2[i] for i in peak_indices])
plt.scatter([td2[i] for i in trough_indices], [d2[i] for i in trough_indices])
print([d2[i] for i in peak_indices])
print([d2[i] for i in trough_indices])

In [ ]:
# finding peaks and troughs for 0.5 V to 3 V

def peakfinder(x,y,height,distance):
  xdata = x
  ydata = y
  # Find indices of peaks and troughs
  peak_indices, peak_properties = find_peaks(ydata, height=height,distance=distance)
  trough_indices, trough_properties = find_peaks(-ydata, height=-height*0.6,distance=distance)
  # trough_indices, trough_properties = find_peaks(-ydata, height=-height,distance=distance)
  # Convert peak indices into time values
  peak_times = []
  for peak in peak_indices:
    peak_time = xdata[peak] # find peak time from index given in 'peaks'
    peak_times.append(peak_time)
  trough_times = []
  for trough in trough_indices:
    trough_time = xdata[trough] # find peak time from index given in 'peaks'
    trough_times.append(trough_time)
  print('Finding peak times...')
  print(peak_times)
  print('Finding trough times...')
  print(trough_times)
  return peak_indices, peak_times, trough_indices, trough_times, xdata, ydata

# reduced distance to increase no. peaks and troughs recognised
distance_minus = 20

# ignored points when calculating def_pp
front = 1
back = 0

heightd = -0.2

peak_def_indices, peak_def_times, trough_def_indices, trough_def_times, xdefdata, ydefdata = peakfinder(td, d,height=heightd, distance = 190)
peak_def_values = ydefdata[peak_def_indices]
trough_def_values = ydefdata[trough_def_indices]

# def_pp = average(peak_def_values[front:(len(peak_def_values)-back)]) - average(trough_def_values[front:(len(peak_def_values)-back)])

# print(def_pp) #peak to peak deflection

lower = 380
upper = 420

tpk = []
pk=[]
ttr=[]
tr=[]

for i in peak_def_indices:
  if ((xdefdata[i]<upper) & (xdefdata[i]>lower)):
    tpk.append(xdefdata[i])
    pk.append(ydefdata[i])

for i in trough_def_indices:
  if ((xdefdata[i]<upper) & (xdefdata[i]>lower)):
    ttr.append(xdefdata[i])
    tr.append(ydefdata[i])

print(tpk)
print(pk)
print(ttr)
print(tr)

def_pp = average(pk)-average(tr)
print(average(pk), average(tr), def_pp)

plt.plot(td, d,color='red') # Plot deflection data
plt.plot(tpk, pk, 'x',label='Peaks',color='green') # Display peaks on plot
plt.plot(ttr, tr, 'x',label='Troughs',color='plum') # Display troughs on plot
plt.xlim(lower, upper)

# plt.xlim(300, 350)

In [ ]:
# finding peaks and troughs for 3 V to 7 V

heightd = -0.2

peak_def_indices, peak_def_times, trough_def_indices, trough_def_times, xdefdata, ydefdata = peakfinder(td2, d2,height=heightd, distance = 190)
peak_def_values = ydefdata[peak_def_indices]
trough_def_values = ydefdata[trough_def_indices]

lower = 10
upper = 40

tpk = []
pk=[]
ttr=[]
tr=[]

for i in peak_def_indices:
  if ((xdefdata[i]<upper) & (xdefdata[i]>lower)):
    tpk.append(xdefdata[i])
    pk.append(ydefdata[i])

for i in trough_def_indices:
  if ((xdefdata[i]<upper) & (xdefdata[i]>lower)):
    ttr.append(xdefdata[i])
    tr.append(ydefdata[i])

print(tpk)
print(pk)
print(ttr)
print(tr)

def_pp = average(pk)-average(tr)
print(def_pp)

plt.plot(td2, d2,color='red') # Plot deflection data
plt.plot(tpk, pk, 'x',label='Peaks',color='green') # Display peaks on plot
plt.plot(ttr, tr, 'x',label='Troughs',color='plum') # Display troughs on plot
plt.xlim(lower, upper)

In [ ]:
av=average([-0.037305376, -0.029993047, -0.031335326, -0.096872263, -0.088220431, -0.094262793, -0.089924939])-average([-0.13321717, -0.138249642, -0.135489386, -0.186438242, -0.188419907, -0.181711631, -0.183630174])
print(av)

In [ ]:
## Au IPMC actuation (Figure S11C)

Volts = [0.5, 1, 1.5, 2, 2.5, 3, 4, 5, 6, 7]
dpp = [0.006724098249999977, 0.015339555199999994, 0.02463648025000001, 0.03379036304761904, 0.0499040922, 0.09703456814285716, 0.17977759820000003, 0.22046214460000002, 0.20879524380000003, 0.21904456754285712]

fig, ax = plt.subplots()
fig.set_figwidth(6)
fig.set_figheight(4)

ax.scatter(Volts, dpp, marker='x', color='green', s=50, lw=1.5, alpha=1, label='Square')
# plt.errorbar(x4, y4, yerr = sd4, fmt='none', color = 'blue')

ax.yaxis.set_major_locator(MultipleLocator(0.08))
ax.yaxis.set_minor_locator(MultipleLocator(0.04))
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))

ax.set_ylim(0,0.25)
ax.set_xlim(0,7.3)
ax.set_xlabel('Voltage amplitude/V')
ax.set_ylabel(r'$\theta_{pp}$/rad')
# ax.axvspan(0, 2, color='y', alpha=0.2, lw=0)

# ax.get_figure().savefig('AuIPMC.svg',format='svg')